#Data 02: Lakers data
로스앤젤레스 레이커스(LA Lakers)는 미국 캘리포니아주 로스앤젤레스를 연고로 하는 NBA 서부 콘퍼런스 퍼시픽 디비전 소속 프로농구 팀으로, NBA 우승 17회를 기록한 명문팀입니다.  
이 데이터는 2008-2009 시즌의 LA 레이커스 경기에 대한 기록입니다.

Data from: http://www.basketballgeek.com/data/  

##1.데이터 둘러보기

In [ ]:
#한글 폰트 설정하기
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf
#런타임 다시 시작

In [ ]:
#기본 패키지 불러오기
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('seaborn')
sns.set(font_scale=1)
plt.rc('font', family='NanumBarunGothic') 
plt.rcParams['font.family'] = 'NanumGothic'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#구글 드라이브 마운트하기(이미 되어있다면 건너뛴다)
from google.colab import drive 
drive.mount('/content/drive')

In [ ]:
#데이터 불러오기
#드라이브에 올려준 csv 데이터 파일 불러오기
lakers = pd.read_csv('/content/drive/MyDrive/Python/02_Lakers/Lakers.csv')

In [ ]:
#불러온 데이터 일단 보기
lakers.head(5)

<컬럼 설명>
* Date: 경기 일자  
* Opponent: 대전 팀  
* Game type: 홈경기 vs. 원정경기  
* Time: 분 : 초  
* Period: 쿼터(한 쿼터당 12분 씩, 동점일 경우 5 쿼터 진행)
* Etype: 유형(ejection / foul / free throw / jump ball / rebound / shot / sub / timeout / turnover / violation
퇴장 / 파울 / 자유투/ 점프볼/ 리바운드/ 슛/ 패스 / 타임아웃/ 턴오버/ 반칙)
* Team: 팀 구분(LAL: LA Lakers, 상대팀)  
* Player: 선수명  
* Result: 결과  
* Points: 점수  
* Type: 세부행동  
* X, Y: 상대편 팀 골대 뒤에서 바라본 X, Y 좌표. 골대의 위치는 (25, 5.25)이다.

In [ ]:
#데이터 기본 정보 확인하기
lakers.info()

* 범주형 변수: *빈도frequency 계산 가능*
               opponent, game_type, period, etype, team, player, resuts, type
* 연속형 변수: *평균, 표준편차, min/max 등 수치 계산 가능*
               시간 변수(date, time), result, x, y

In [ ]:
#연속형(수치) 데이터의 통계치 확인하기
lakers.describe()

In [ ]:
#범주형 데이터의 통계치 확인하기
lakers.describe(include=np.object)

In [ ]:
#변수형 컬럼 안에는 어떤 내용이 들어있을까?
print('Oppenent: ', lakers['opponent'].unique())
print('Game type: ', lakers['game_type'].unique())
print('Period: ', lakers['period'].unique())
print('Etype: ', lakers['etype'].unique())
print('Team: ', lakers['team'].unique())
print('Result: ', lakers['result'].unique())
print('Points: ', lakers['points'].unique())
print('Type: ', lakers['type'].unique())

In [ ]:
#데이터 내용 간단히 정리해서 출력하기
print("총 데이터 개수: ", lakers.shape[0]*lakers.shape[1])
print("총 결측치 수: {} = 전체 데이터의 {:.2f}% ".format(lakers.isnull().sum().sum(), (lakers.isnull().sum().sum()*100)/(lakers.shape[0]*lakers.shape[1])))
print("LA 레이커스와 경기한 팀 수: ", lakers['opponent'].nunique()-1)
print("경기에 등장하는 행동 수: ", lakers['etype'].nunique())
print("경기에 등장하는 세부행동 수: ", lakers['type'].nunique())

##2.데이터에서 질문 만들기

* LA레이커스의 홈 경기 vs. 원정경기 비율은?
* 경기에서 선수들이 가장 많이 하는 행동유형(etype)은?
* 이번 시즌에서 LA레이커스의 경기 결과는?
* LA레이커스 선수들은 코트의 어느 위치에서 어떤 동작을 했는가?

##3.데이터 정비하기

###3-1. 시간 타입의 컬럼 정비하기

In [ ]:
#date 컬럼은 int64, time 컬럼은 object 타입이므로 date 컬럼의 타입을 변환한 후 두 컬럼을 합친다
lakers['date'] = lakers['date'].apply(str)
lakers['datetime'] = lakers['date'] + ' ' + lakers['time']
lakers.head()

In [ ]:
#date 컬럼과 새로 만든 datetime 컬럼을 날짜 타입으로 바꿔준다
#날짜 타입은 연, 월, 일, 시, 분, 초 등의 정보를 보기 편하다(시계열 분석시 편리!)
lakers['datetime'] = pd.to_datetime(lakers['datetime'],format='%Y%m%d %H:%M:%S')
lakers['date'] = pd.to_datetime(lakers['date'],format='%Y%m%d')
lakers.head()

###3-2. 결측치 처리하기

In [ ]:
#NULL이 존재하는 컬럼 확인하기 
lakers.isnull().any(axis=0)
#player, result, type, x, y 컬럼에 NaN 존재한다

In [ ]:
#컬럼의 특성마다 어떻게 null을 처리해줄 것인가?
#player, result, type은 object 값이 들어가 있으므로 굳이 NaN에 값을 채울 필요 없다
#x, y 컬럼의 NaN: 그냥 둬도되지만 0을 채워 넣어보자
#0을 채워 넣기 전에, 실제 데이터에 0이 존재하는지 확인!
print(lakers.index[lakers['x'] == '0'].tolist())
print(lakers.index[lakers['y'] == '0'].tolist())
#존재하지 않는다!

In [ ]:
#x, y 포지션의 NaN에 0 채워넣기
lakers['x'].fillna(0, inplace=True)
lakers['y'].fillna(0, inplace=True)
lakers.head()

##4.EDA & Visualizaion

###4-1. 홈 경기 대 원정 경기의 비율

In [ ]:
#날짜별로 홈 경기인지 원정경기인지 보려면?
lakers[['date','game_type']].groupby(['date','game_type']).sum()

In [ ]:
#reset_index 함수로 정리해주기
lakers[['date','game_type']].groupby(['date','game_type']).sum().reset_index()

In [ ]:
#홈 경기 대 원정 경기의 비율: 
#날짜별로 home/away 데이터가 중복되므로 처음 하나만 남기고 날려버려서 해결하는 방법도 있다
df1 = lakers.drop_duplicates(['date'], keep='first')
df1

In [ ]:
#홈 경기 대 원정 경기의 비율: 그래프로 나타내기
f, ax = plt.subplots(1, 2, figsize=(18, 8))

#pie plot
df1['game_type'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True)
ax[0].set_title('Pie plot: 홈 경기 vs. 원정 경기', size = 15)
ax[0].set_ylabel('') #y 라벨 없애기

#count plot
sns.countplot('game_type', data=df1, ax=ax[1])
ax[1].set_title('Count plot: 홈 경기 vs. 원정 경기', size = 15)

plt.show()

###4-2.경기에서 발생한 행동 유형 확인하기

In [ ]:
#몇개의 행동 유형이 있는가?
print('Etype: ', lakers['etype'].unique())

In [ ]:
#모든 경기를 포함해서 각각의 행동유형이 몇번 발생했는지 세기
lakers['etype'].value_counts()

In [ ]:
#행동유형 발생 횟수 그래프로 나타내기
f, ax = plt.subplots(1, 1, figsize=(14, 6))

sns.countplot(x=lakers['etype'], ax=ax)
ax.set_title('행동 유형 발생 횟수', size=15)
ax.set_ylabel('')

plt.show()

In [ ]:
#특정 쿼터에서 많이 발생한 행동이 있을까?
#groupby를 사용해서 각 쿼터별로 각각의 행동이 몇 회 발생했는지 세 보기
df2 = lakers.groupby(['etype','period']).size().reset_index().pivot(columns='period',index='etype', values=0)
df2

In [ ]:
#stack plot 그리기
#어떤 컬럼을 stack 할 것인가? >> 행동유형은 종류가 많다. 쿼터 별로 나누자!
df2.plot(kind='bar', stacked=True, figsize=(18, 6))
plt.title('쿼터별 행동 발생 횟수', size=15)
plt.xticks(rotation=30)

plt.show()

###4-3. 경기 결과 집계하고 시계열 그래프로 그리기

In [ ]:
#날짜별 경기 결과를 group by로 계산해보자
#결과 컬럼에서 성공했을때의 득점만 더하면 경기의 총 점수가 나온다
made = lakers['result'].isin(['made'])
lakers[made].head()

In [ ]:
#날짜별 팀별 슛 성공 건수를 집계해보면?
df3 = lakers[made][['date', 'team', 'points']].groupby(['date', 'team'], as_index=True).sum()
df3

In [ ]:
#reset index 사용
df3.reset_index(inplace=True)
df3

In [ ]:
#LA 레이커스와 상대편으로 경기 결과 나누기
lal = df3[df3['team'] == 'LAL']
opp = df3[df3['team'] != 'LAL']

In [ ]:
lal

In [ ]:
#경기 결과를 가로 방향으로 붙이기
#pd.merge 함수 사용: 붙일 때 key는? = 날짜!
df4 = pd.merge(lal, opp, on='date')
df4

In [ ]:
#La레이커스가 이긴 날과 진 날을 고르자
win = df4[df4['points_x'] > df4['points_y']]
los = df4[df4['points_x'] < df4['points_y']]
win.shape[0]     
print("08 - 09 시즌에 LA 레이커스는 총 {}회의 경기 중, {}번 우승했다".format(df4.shape[0], df4[df4['points_x'] > df4['points_y']].shape[0] ))                         

In [ ]:
#경기 결과를 시계열 그래프로 확인하기
plt.figure(figsize=(16,5))

plt.title("08 - 09 Season : LA Lakers", fontsize=15)

#LA 레이커스 점수를 오렌지 실선으로 나타낸다
plt.plot(lal['date'], lal["points"], "-", alpha= 1, color = 'orange', label='LA Lakers')

#LA 레이커스가 이긴 날은 빨간 점, 진 날은 파란 점으로 표시해주자
plt.plot(win['date'], win["points_x"], 'o', alpha=.6, color = 'red')
plt.plot(los['date'], los["points_x"], 'o', alpha=.6, color = 'blue')

#상대팀 점수를 초록색 점선으로 나타낸다
plt.plot(opp['date'], opp["points"], "--", alpha=.6, color = 'green', label='Opposite team')

plt.legend(loc='upper left', borderaxespad=0, fontsize=13)
plt.xticks(rotation=45)

plt.show()

###4-4. LA레이커스 팀의 코트 위치별 행동

In [ ]:
#LA lakers의 데이터만 가져오기: Team = LAL 인 것
lal2 = lakers[lakers['team'] == 'LAL']
lal2

In [ ]:
#슛한 위치 확인하기
#득점은 어느 위치에서 발생했는가?
sns.set_style('whitegrid')
f, ax = plt.subplots(1, 2, figsize=(12, 7), dpi = 80)

#득점 성공/실패 분포
g1 = sns.scatterplot(data=lal2, x='x', y='y', hue='result', alpha = 0.3, palette='Paired', ax=ax[0])
g2 = sns.kdeplot(data=lal2, x='x', y='y', hue='result', palette='Paired', ax=ax[0])

#득점 점수 별 분포
g3 = sns.scatterplot(data=lal2, x='x', y='y', hue='points', alpha = 0.3, palette='Set3', ax=ax[1])
g4 = sns.kdeplot(data=lal2, x='x', y='y', hue='points', palette='Set3', ax=ax[1])

plt.show()
#x, y의 NaN을 0으로 처리해줬기 때문에 0 근처에 쏠림이 생겼다

In [ ]:
#세부 행동 별 슛 위치는 어떻게 될까?
#가장 많이 한 세부 행동 10개 고르기
lal3 = lal2[lal2['type'].isin(lal2['type'].value_counts().head(10).index)]
lal3

In [ ]:
#세부 행동 별 위치 확인하기

sns.set_style('whitegrid')
f, ax = plt.subplots(1, 1, figsize=(5, 7), dpi = 80)

sns.scatterplot(data=lal3, x='x', y='y', hue=lal3['type'], palette='Set3', ax=ax)

#Legend가 10개이기 때문에 그래프 안에 넣으면 혼란스럽다. 옆으로 빼 주자!
ax.legend(title='Type', bbox_to_anchor= (1.03, 1))

plt.show()

##5.Review

* LA레이커스 데이터는:  
총 데이터 개수:  450112
총 결측치 수: 76625 = 전체 데이터의 17.02% 
LA레이커스와 경기한 팀 수:  28
경기에 등장하는 행동 수:  10
경기에 등장하는 세부행동 수:  73

* LA레이커스의 홈 경기 vs. 원정경기 비율은?  
  : 홈 경기와 원정경기를 1:1 비율로 치렀다.
* 경기에서 선수들이 가장 많이 하는 행동유형(etype)은?
  : 슛을 가장 많이 하고 그 다음으로는 리바운드.
* 이번 시즌에서 LA레이커스의 경기 결과는?  
  : 08 - 09 시즌에 LA 레이커스는 총 78회의 경기 중, 63번 우승했다.
* LA레이커스 선수들은 코트의 어느 위치에서 어떤 동작을 했는가?
  : 상대편 골대 기준으로 왼쪽에서 슛 했을 때 더 많이 성공했다

* 더 분석해볼 점은?  
  : x, y 좌표의 NaN에 0을 넣지 않는 것이 분석 결과에 더 좋다!  
  : 특정 선수의 행동 패턴 분석
